# Your name: Adrien Kahn

# Gaussian mixture model: EM vs VI


We consider two algorithms for inference in the Gaussian mixture model: expectation maximization (EM) and mean-field variational inference (VI).

In both cases, we deal with a mixture of $K$ real-valued normalized Gaussian random variables. We assume that $K$ is known and for simplicity that the variances of the Gaussian distributions are all equal to one.There are $n$ observations denoted by $x=(x_1,\dots, x_n)\in \mathbb{R}^n$. The weights of the mixture as well as the mean of each Gaussian distribution is not known and need to be estimated.

Note: you can read Sections 1 and 2 independently (as a result some notations are repeated).

## 1. Expectation Maximization

**setting:**
- $\mu=(\mu_1,\dots, \mu_K)\in \mathbb{R}^K$
- $p(c=k) = \pi_k$ with $\pi =(\pi_1,\dots, \pi_K)$ and $\sum_k \pi_k =1$, $\pi_k\geq 0$
- parameter $\theta=(\mu,\pi)$
- Gaussian distribution with normalized variance and mean $m$: $f_m(x) = \frac{1}{\sqrt{2\pi}}\exp\left(-\frac{1}{2}(x-m)^2 \right)$.
- Gaussian mixture model: for $x=(x_1,\dots, x_n)\in \mathbb{R}^n$ and $c = (c_1,\dots c_n)\in [K]^n$, we have $p_{\theta}(x,c) = \prod_{i=1}^n \pi_{c_i} f_{\mu_{c_i}}(x_i)$

Likelihood (we observe only $x$ and not the classes given by $c$)
\begin{align*}
\ell(\theta) &= \ln p_\theta(x) = \ln \sum_{c\in [K]^n} p_\theta(x,c)
\end{align*}
is not tractable because the time complexity for evaluating the sum is $O(K^n)$.

>Why can't we write:
$$ \ell(\theta) = \sum_{i = 1}^n \ln p_\theta(x_i) = \sum_{i = 1}^n \ln \left( \sum_{c_i = 1}^K p_\theta(x_i, c_i) \right) $$
Then the complexity is only $O(nK)$.

**ELBO**
\begin{align*}
\ell(\theta) &= \sum_{c\in [K]^n} q(c) \ln  p_\theta(x)\\
&= \sum_{c\in [K]^n} q(c) \ln \frac{p_\theta(x,c)q(c)}{p_{\theta}(c|x)q(c)}\\
&= \sum_{c\in [K]^n} q(c) \ln \frac{p_\theta(x,c)}{q(c)} + \sum_{c\in [K]^n} q(c) \ln \frac{q(c)}{p_{\theta}(c|x)}\\ 
&= \text{ELBO}(q)+\text{KL}\left(q(c)\| p_\theta(c|x)\right)
%&= \sum_{c\in [K]^n} q(c|x) \ln p_\theta(x,c) + H(q(c|x))
%&\geq \sum_{c\in [K]^n} q(c|x) \ln p_\theta(x,c) 
\end{align*}

Note that for $q(c) = p_{\theta}(c|x)$, we get $\text{KL}\left(p_{\theta}(c|x)\| p_\theta(c|x)\right)=0$, so that $\text{ELBO}(p_{\theta}(c|x)) =\ell(\theta)$.
Now for an estimate $\theta_t$ of the parameter, we take $q(c) = p_{\theta_t}(c|x)$ and define
\begin{align*}
\ell(\theta|\theta_t)\stackrel{\Delta}{=} \sum_{c\in [K]^n} p_{\theta_t}(c|x) \ln \frac{p_\theta(x,c)}{p_{\theta_t}(c|x)}, %= \text{ELBO}( p_{\theta_t}(c|x)),
\end{align*}
so that by previous argument, we have
\begin{align*}
\ell(\theta|\theta_t) = \ell(\theta)-\text{KL}\left(p_{\theta_t}(c|x)\| p_\theta(c|x)\right),
\end{align*}
in particular, we see that $\ell(\theta|\theta_t) \leq  \ell(\theta)$ and $\ell(\theta_t|\theta_t) = \ell(\theta_t)$.

**EM algorithm:**
- E-step: compute $p_{\theta_t}(c|x)$
- M-step: maximize in $\theta$, $\theta_{t+1}=\arg\max_{\theta}\ell(\theta|\theta_t)=\arg\max_{\theta}\sum_{c\in [K]^n} p_{\theta_t}(c|x) \ln {p_\theta(x,c)}$

**Properties of EM:** the sequence of log-likelihoods $(\ell(\theta_t))_{t\geq 0}$ is non-decreasing and thus converges.

The proof is easy: note that by applying the equality above for $\theta=\theta_{t+1}$, we get
\begin{align*}
\ell(\theta_{t+1}|\theta_t) = \ell(\theta_{t+1})-\text{KL}\left(p_{\theta_t}(c|x)\| p_{\theta_{t+1}}(c|x)\right)\leq \ell(\theta_{t+1}),
\end{align*}
so that we have
\begin{align*}
\ell(\theta_{t+1}) \geq \ell(\theta_{t+1}|\theta_t) \geq \ell(\theta_{t}|\theta_t) = \ell(\theta_t)
\end{align*}

We now need to derive the EM algorithm for the GMM.

**Question 1**
Show that for a parameter $\theta =(\mu, \pi)$ and $x\in \mathbb{R}^n$, we have:
\begin{align*}
p_{ik}(\theta,x) &\stackrel{\Delta}{=} p_{\theta}(c_i=k|x_i)\\
& = \frac{\pi_k(\theta) \exp\left( -\frac{1}{2}(x_i-\mu_k(\theta))^2\right)}{\sum_\ell \pi_\ell(\theta) \exp\left( -\frac{1}{2}(x_i-\mu_\ell(\theta))^2\right)},
\end{align*}
with a slight abuse of notation where $\theta= (\mu(\theta),\pi(\theta))$.

> \begin{align}
p_{\theta}(c_i=k|x_i) &= \frac{p_{\theta}(c_i=k, x_i)}{p_{\theta}(x_i)} \\
&= \frac{p_{\theta}(c_i=k, x_i)}{\sum_\ell p_{\theta}(c_i = \ell, x_i)} \\
&= \frac{\pi_k(\theta) \exp\left( -\frac{1}{2}(x_i-\mu_k(\theta))^2\right)}{\sum_\ell \pi_\ell(\theta) \exp\left( -\frac{1}{2}(x_i-\mu_\ell(\theta))^2\right)}
\end{align}

**Question 2**
We define
\begin{align*}
n_{k}(\theta, x) = \sum_{i=1}^n p_{ik}(\theta,x).
\end{align*}

Derive the **EM for Gaussian mixture**

- **Expectation** given $\theta_t$ and the samples $x=(x_1,\dots, x_n)$, compute for all $i\in [n]$ and $k\in [K]$:

\begin{align*}
p_{ik}(\theta_t,x) & = \frac{\pi_k(\theta) \exp\left( -\frac{1}{2}(x_i-\mu_k(\theta))^2\right)}{\sum_\ell \pi_\ell(\theta) \exp\left( -\frac{1}{2}(x_i-\mu_\ell(\theta))^2\right)}
\end{align*}

- **Maximization** given $\theta_t$ and the samples $x=(x_1,\dots, x_n)$, the parameter $\theta_{t+1}$ maximizing $\ell(\theta|\theta_t)$ is given by

\begin{align*}
\pi_k(\theta_{t+1}) &= \frac{n_k(\theta_t, x)}{\sum_\ell n_\ell(\theta_t, x)} \\
\mu_k(\theta_{t+1}) &= \frac 1 {n_k(\theta_t, x)} \sum_i x_i p_{ik} (\theta_t, x)
\end{align*}

> Question 1 gives the formula for the Expectation part.
>
> For the maximization part, we must maximize $\ell(\theta | \theta_t)$ for $\theta$, subject to $\sum_k \pi_k = 1$. We therefore introduce a Lagrange multiplier $\lambda$ in the function to minimize:
\begin{align}
g(\theta) &= \sum_c p_{\theta_t}(c|x) \ln p_\theta(c,x) - \lambda \left(\sum_k \pi_k - 1\right)\\
&= \sum_c p_{\theta_t}(c|x) \sum_i \left[ \ln \pi_{c_i} + \ln \left( f_{\mu_{c_i}}(x_i) \right) \right] - \lambda \left(\sum_k \pi_k - 1\right)
\end{align}
> The derivatives give:
\begin{align}
\frac{\partial g}{\partial \pi_k} &= \sum_c p_{\theta_t}(c|x) \sum_i \frac{\delta_{c_i, k}}{\pi_k} - \lambda \\
&= \frac 1 {\pi_k} \sum_i \sum_c p_{\theta_t}(c|x) \delta_{c_i, k} - \lambda\\
&= \frac 1 {\pi_k} \sum_i \sum_{c, c_i = k} p_{\theta_t}(c|x) - \lambda\\
&= \frac 1 {\pi_k} \sum_i p_{\theta_t}(k|x_i) \prod_{j \neq i} \underbrace{\sum_{c_j} p_{\theta_t}(c_j|x_j)}_{=1} - \lambda\\
&= \frac {n_k(\theta_t, x)} {\pi_k} - \lambda
\end{align}
and
\begin{align}
\frac{\partial g}{\partial \mu_k} &= \sum_c p_{\theta_t}(c|x) \sum_i \delta_{c_i, k} (x_i - \mu_k) \\
&= \sum_i (x_i - \mu_k) \sum_{c, c_i = k} p_{\theta_t}(c|x)\\
&= \sum_i (x_i - \mu_k) p_{\theta_t}(k|x_i) \prod_{j \neq i} \underbrace{\sum_{c_j} p_{\theta_t}(c_j|x_j)}_{=1} \\
&= \sum_i (x_i - \mu_k) p_{ik} (\theta_t, x)
\end{align}
>
> Setting both derivatives to zero we obtain expressions for $\pi_k(\theta_{t+1})$ and $\mu_k(\theta_{t+1})$:
>
>\begin{align}
\pi_k(\theta_{t+1}) &= \frac{n_k(\theta_t, x)}{\sum_\ell n_\ell(\theta_t, x)} \\
\mu_k(\theta_{t+1}) &= \frac 1 {n_k(\theta_t, x)} \sum_i x_i p_{ik} (\theta_t, x)
\end{align}

**Question 3**
Compute $\ell(\theta_t)=\ell(\theta_t|\theta_t)$.

## 2. Mean-field Variational Inference

**setting:**
- $\mu=(\mu_1,\dots, \mu_K)\in \mathbb{R}^K$ with prior $p(\mu_k)=\mathcal{N}(\mu_k|0,\sigma^2)$ and $p(\mu)= \prod_k p(\mu_k)$.
- $p(c=k) = \pi_k$ with $\pi =(\pi_1,\dots, \pi_K)$ and $\sum_k \pi_k =1$, $\pi_k\geq 0$ with a Dirichlet prior $p(\pi)=C(\alpha)\prod_{k=1}^K\pi_k^{\alpha-1}$, with $C(\alpha) = \frac{\Gamma(K\alpha)}{\Gamma(\alpha)^K}$.
- Gaussian distribution with normalized variance and mean $m$: $f_m(x) = \frac{1}{\sqrt{2\pi}}\exp\left(-\frac{1}{2}(x-m)^2 \right)$.
- Gaussian mixture: for $x=(x_1,\dots, x_n)\in \mathbb{R}^n$ and $c = (c_1,\dots c_n)\in [K]^n$, we have $p(x,c,\mu,\pi) = p(\mu)p(\pi)\prod_{i=1}^n \pi_{c_i} f_{\mu_{c_i}}(x_i)$

Hence, the evidence is
\begin{align*}
p(x) = \sum_{c\in [K]^n} \int_{\mu,\pi} p(\mu)p(\pi)\prod_{i=1}^n \pi_{c_i} f_{\mu_{c_i}}(x_i) d\mu d\pi = \int_{\mu,\pi} p(\mu)p(\pi)\prod_{i=1}^n\sum_{c_i\in[K]} \pi_{c_i} f_{\mu_{c_i}}(x_i) d\mu d\pi
\end{align*}

**ELBO**


\begin{align*}
\ln p(x) &= \sum_c\int_{\mu,\pi} q(\mu,\pi,c)\ln \frac{p(x,c,\mu,\pi)}{p(c,\mu,\pi|x)} d\mu d\pi\\
&= \sum_c\int_{\mu,\pi} q(\mu,\pi,c)\ln \frac{p(x,c,\mu,\pi)}{q(\mu,\pi,c)}-\sum_c\int_{\mu,\pi} q(\mu, \pi,c)\ln \frac{p(c,\mu, \pi|x)}{q(\mu, \pi,c)}\\
&= \text{ELBO}(q)+\text{KL}\left( q(\mu,\pi,c)\| p(c,\mu,\pi|x)\right)
\end{align*}

**Maximizing ELBO with a mean-field distribution**

We are considering now factorized distributions for $q$:
\begin{align*}
q(\mu,\pi,c) = q(\mu,\pi)q(c).
\end{align*}

We have for the factor $q(\mu,\pi)$:
\begin{align*}
\text{ELBO}(q) &= \int_{\mu, \pi} q(\mu,\pi)\sum_{c\in [K]^n} q(c)\ln p(x, c,\mu,\pi)d\mu d\pi +H(q(\mu,\pi))+H(q(c))\\
&=\int_{\mu,\pi} q(\mu,\pi)\mathbb{E}_{q(c)}\left[\ln p(x, c,\mu,\pi)\right]d\mu d\pi-\int_{\mu,\pi} q(\mu,\pi)\ln q(\mu,\pi)d\mu d\pi +\text{cst}
\end{align*}

**Question 4** 
When $q(c)$ is fixed (we denote $\varphi_{i}(k) = q(c_i=k)$ the $i$-th marginal of $q$), find the distirbution $q^*(\mu,\pi)$ maximizing ELBO.
(Hint: if you are not familiar with the Dirichlet distribution, have a look at its [definition](https://en.wikipedia.org/wiki/Dirichlet_distribution))

**Question 5**
Similarly maximizing ELBO with $q(\mu,\pi)$ fixed, give the distribution $q^*(c)$. (Hint: the log moment of the [Dirichlet distribution](https://en.wikipedia.org/wiki/Dirichlet_distribution) are given by the [digamma function](https://en.wikipedia.org/wiki/Digamma_function)).

**Question 6** Derive the mean-field VI algorithm.

**Question 7** To compute the ELBO,  we use the following decomposition
\begin{align*}
\text{ELBO}(q) = \underbrace{\mathbb{E}_{q(\mu)q(\pi)}\ln p(\mu)p(\pi)}_{A}+\underbrace{\mathbb{E}_{q(\mu)q(\pi)q(c)}\ln \prod_{i=1}^n\pi_{c_i}f_{\mu_{c_i}}(x_i)}_{B} +H(q(\mu))+H(q(\pi))+H(q(c)).
\end{align*}
Compute each term.

**Question 8** Consider the **Uniformative priors** by taking $\sigma^2\to\infty$ and $\alpha \to 0$ and use the fact that as $z\to\infty$, we have $\psi(z) \sim \ln(z)-\frac{1}{2z}$ to show the similarity between EM and VI.

# 3. Code

We now code both algorithms.

The code below, will produce data for a simple mixture of 2 Gaussian distributions.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy import stats
from scipy.special import psi, gamma

np.random.seed(654)
# Draw samples from two Gaussian w.p. c_i ~ Bernoulli(p)
p = 0.75
ps = [p, 1-p]
mus = [2, 5]
generative_m = np.array([stats.norm(mus[0], 1), stats.norm(mus[1], 1)])
c_i = stats.bernoulli(p).rvs(100)
x_i = np.array([g.rvs() for g in generative_m[c_i]])

# plot generated data and the latent distributions
x = np.linspace(-5, 12, 150)
plt.figure(figsize=(16, 6))
plt.plot(x, (1-p)*generative_m[0].pdf(x))
plt.plot(x, p*generative_m[1].pdf(x))
plt.plot(x, (1-p)*generative_m[0].pdf(x) + p*generative_m[1].pdf(x), lw=1, ls='-.', color='black')
plt.fill_betweenx((1-p)*generative_m[0].pdf(x), x, alpha=0.1)
plt.fill_betweenx(p*generative_m[1].pdf(x), x, alpha=0.1)
plt.vlines(x_i, 0, 0.01, color=np.array(['C0', 'C1'])[c_i])

General rule: try to avoid for loop and use broadcasting as much as possible. It makes your code more efficient and less prone to errors...

In [3]:
# Here are some examples using broadcasting:
z = np.random.normal(size=(2,5))
w = np.random.normal(size=2)

In [4]:
z.shape

(2, 5)

In [5]:
w.shape

(2,)

In [6]:
(w[:,np.newaxis]).shape

(2, 1)

In [7]:
(z*w[:,np.newaxis]).shape

(2, 5)

In [8]:
# do not do this:
res = np.zeros(2)
for i in range(2):
    res[i] = np.sum(z[i,:]*w[i])
res

array([ 0.02034857, -0.02583676])

In [9]:
# instead do this:
(z*w[:,np.newaxis]).sum(1)

array([ 0.02034857, -0.02583676])

In [14]:
(z*w[:,np.newaxis]).sum(0)

array([-0.79386331,  1.46279674, -0.72906848,  1.06800665, -1.01335978])

## 3.1 EM algorithm

Complete the code below.

- **Expectation** given $\theta_t$ and the samples $x=(x_1,\dots, x_n)$, compute for all $i\in [n]$ and $k\in [K]$:

\begin{align*}
p_{ik}(\theta_t,x) & = \frac{\pi_k(\theta) \exp\left( -\frac{1}{2}(x_i-\mu_k(\theta))^2\right)}{\sum_\ell \pi_\ell(\theta) \exp\left( -\frac{1}{2}(x_i-\mu_\ell(\theta))^2\right)}
\end{align*}

- **Maximization** given $\theta_t$ and the samples $x=(x_1,\dots, x_n)$, the parameter $\theta_{t+1}$ maximizing $\ell(\theta|\theta_t)$ is given by

\begin{align*}
\pi_k(\theta_{t+1}) &= \frac{n_k(\theta_t, x)}{\sum_\ell n_\ell(\theta_t, x)} \\
\mu_k(\theta_{t+1}) &= \frac 1 {n_k(\theta_t, x)} \sum_i x_i p_{ik} (\theta_t, x)
\end{align*}

In [93]:
class EM:
    def __init__(self, k):
        self.k = k # number of clusters
        self.n = None
        self.mu = None # (k,)
        self.p_ik = None # (k,n)
        self.pi = None # (k,)
        self.lb = [] # to keep track of the lower bound
        self.x = None # to store the data

    def expectation_step(self):
        # Update p_ik with the current pi and mu
        self.p_ik = self.pi[:,np.newaxis] * np.exp(-(self.x[np.newaxis,:] - self.mu[:,np.newaxis])**2 / 2)
        self.p_ik /= self.p_ik.sum(0)[np.newaxis,:]
        
    def maximization_step(self):
        # Update pi and mu with the current p_ik
        self.pi = self.p_ik.sum(1)
        self.pi /= self.pi.sum()
        self.mu = (self.x[np.newaxis,:] * self.p_ik).sum(1) / self.p_ik.sum(1)
        
    def lower_bound(self):
        #
        # you code computing the lower bound
        #
        return -self.n/2*np.log(2*math.pi)#+your code

    def fit(self, x):
        # once other methods are implemented, this should work fine!
        self.x = x
        self.n = x.shape[0]
        self.mu = np.random.uniform(x.min(), x.max(), size=self.k)
        self.p_ik = np.zeros((self.k, self.n))
        self.pi = np.ones(self.k) / self.k

        i = -1
        while (i<6 or ~np.isclose(self.lb[i-5], self.lb[i])):
            self.expectation_step()
            self.maximization_step()
            self.lb.append(self.lower_bound())
            i+=1
        return self.lb

In [94]:
em = EM(2)
lb = em.fit(x_i)

In [95]:
# plot the lower bound
plt.plot(lb)

In [96]:
# this should display the fitted distribution
fitted_m = [stats.norm(mu, 1) for mu in em.mu]
fitted_p = em.pi

if fitted_p[0] > fitted_p[1]:
    fitted_p = fitted_p[::-1]
    fitted_m = fitted_m[::-1]

plt.figure(figsize=(16, 6))
plt.vlines(x_i, 0, 0.01, color=np.array(['C0', 'C1'])[c_i])
plt.plot(x, fitted_p[0]*fitted_m[0].pdf(x))
plt.plot(x, fitted_p[1]*fitted_m[1].pdf(x))
plt.plot(x, (1-p)*generative_m[0].pdf(x), color='black', lw=1, ls='-.')
plt.plot(x, p*generative_m[1].pdf(x), color='black', lw=1, ls='-.')

## 3.2 VI algorithm

Complete the code below.

In [ ]:
# these functions should be useful to compute the ELBO
def my_beta(b):
    return gamma(np.sum(b))/np.prod(gamma(b))

def my_C(a,k):
    return gamma(k*a)/np.float_power(gamma(a),k)

In [ ]:
class VI:
    def __init__(self,k, prec=0.0001, alpha=0.0001):
        self.k = k
        self.n = None 
        self.means = None # (k,)
        self.vars = None # (k,)
        self.phi = None # (k,n)
        self.beta = None # (k,)
        self.rho = None # (k,)
        self.norm_rho = 0 # see definition in function fit
        self.prec = prec # precision for the Gaussian prior
        self.alpha= alpha # alpha for the Dirichlet prior
        self.lb = [] # to keep track of the lower bound
        self.x = None # to store the data
        
    def update_mu_pi(self):
        #
        # your code here
        #
        pass
                        
    def update_phi_rho(self):
        #
        # your code here
        #
        pass
    
    def lower_bound(self):
        entrop1 = self.k/2+0.5*np.sum(np.log(2*math.pi*self.vars))
        #
        # your code here to compute ELBO
        #
        return entrop1#+your code here
    
    def fit(self, x):
         # once other methods are implemented, this should work fine!
        self.n = x.shape[0]
        self.x = x
        self.means = np.random.uniform(x.min(), x.max(), size=self.k)
        self.vars = np.ones(self.k)
        self.phi = np.random.uniform(0,1,size=((self.k,self.n)))
        self.phi /= self.phi.sum(0)
        self.beta = (self.n/self.k+self.alpha)*np.ones(self.k)
        self.norm_rho = np.exp(-psi(self.k*self.alpha+self.n))
        
        i = -1
        while ( i<6 or ~np.isclose(self.lb[i-5], self.lb[i])):
            self.update_phi_rho()
            self.update_mu_pi()
            self.lb.append(self.lower_bound())
            i +=1
        return self.lb
 

In [ ]:
vi = VI(2)
lb_vi = vi.fit(x_i)

In [ ]:
#plot the ELBO
plt.plot(lb_vi)

In [ ]:
# this should display the fitted distribution
fitted_m_vi = [stats.norm(mu, 1) for mu in vi.means]
fitted_p_vi = vi.phi.mean(axis=1)

if fitted_p_vi[0] > fitted_p_vi[1]:
    fitted_p_vi = fitted_p_vi[::-1]
    fitted_m_mi = fitted_m_vi[::-1]

plt.figure(figsize=(16, 6))
plt.vlines(x_i, 0, 0.01, color=np.array(['C0', 'C1'])[c_i])
plt.plot(x, fitted_p_vi[0]*fitted_m_vi[0].pdf(x))
plt.plot(x, fitted_p_vi[1]*fitted_m_vi[1].pdf(x))
plt.plot(x, (1-p)*generative_m[0].pdf(x), color='black', lw=1, ls='-.')
plt.plot(x, p*generative_m[1].pdf(x), color='black', lw=1, ls='-.')